In [1]:
import cv2
import numpy as np

# read original BRG image
image = cv2.imread('files/shapes_leo.jpg',1)
# convert to grayscale for threshold operation
image_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
# convert image to black and white
ret,image_bw = cv2.threshold(image_gray, 125, 255, cv2.THRESH_BINARY_INV)
# create kernel of erosion/dilation
kernel = np.ones((10,10),np.uint8)
# Close to simplify background 
closing = cv2.morphologyEx(image_bw, cv2.MORPH_CLOSE, kernel)
# convert back to BRG
closing = cv2.cvtColor(closing,cv2.COLOR_GRAY2BGR)
# get the original image without the background
mask = closing.copy()
masked_image=cv2.add(image,mask)
# convert to grayscale for Hough transform
masked_image_gray = cv2.cvtColor(masked_image,cv2.COLOR_BGR2GRAY)
# Gaussian blur for making edges smooth and remove high-frequency noise
blurred_image = cv2.GaussianBlur(masked_image_gray, (7,7), 0)
# perform Hough transform for detecting circles
circles  = cv2.HoughCircles(blurred_image, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=10, maxRadius=100)

if circles is None:
        
    print('No circles detected')
        
else:

    # Trace circles on processed image
    image_proc = masked_image.copy()
    DetectedCircles = False
    for i in circles[0,:]:
    
        if i[2]<10: 
    
            # print radius less than 10 message
            print('Circle has radius less than 10')
            continue
                
        else:
        
            DetectedCircles = True
            
            #draw the outer circle
            cv2.circle(image_proc, (i[0], i[1]), i[2], (255,0,0), 2)
    
            #draw the center of the circle
            cv2.circle(image_proc, (i[0], i[1]), 2, (255,0,0), 3)

            
if DetectedCircles is True:
      
        image_out = np.concatenate((image, image_proc), axis=1)
        cv2.imshow('Original and Processed Images',image_out)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
elif DetectedCircles is False:
    
    print('No circle with radius greater than 10 has been found')
